<a href="https://colab.research.google.com/github/xXxmemoxXx/README/blob/main/prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [ ]:
pip install pymysql

In [ ]:
# @title
!pip install mysql-connector-python pandas matplotlib seaborn

In [ ]:

import plotly.graph_objects as go
import plotly.io as pio

import mysql.connector
import pandas as pd
import plotly.graph_objects as go

# Conexión a MySQL
conn = mysql.connector.connect(
    host="miaa.mx",
    user="miaamx_telemetria",
    password="eORPvq4.yy@C",
    database="miaamx_telemetria"
)

# Crear un cursor
cursor = conn.cursor()

# Consulta para obtener datos de ambos GATEID en las últimas 24 horas
query = """
    SELECT FECHA, GATEID, VALUE
    FROM vfitagnumhistory
    WHERE GATEID IN (66482, 66484)
    AND FECHA >= NOW() - INTERVAL 7 DAY
    ORDER BY FECHA ASC
"""
cursor.execute(query)

# Obtener los datos
datos = cursor.fetchall()

# Cerrar la conexión
cursor.close()
conn.close()

# Convertir los datos en un DataFrame de Pandas
df = pd.DataFrame(datos, columns=["FECHA", "GATEID", "VALUE"])

# Convertir la columna FECHA a tipo datetime
df["FECHA"] = pd.to_datetime(df["FECHA"])

# Separar datos por GATEID
df_caudal = df[df["GATEID"] == 66482]
df_presion = df[df["GATEID"] == 66484]

# Crear la figura con Plotly
fig = go.Figure()

# Agregar la línea de caudal (azul)
fig.add_trace(go.Scatter(
    x=df_caudal["FECHA"], y=df_caudal["VALUE"], mode="lines+markers",
    name="Caudal (L/s)", line=dict(color="blue"), marker=dict(size=6),
    hovertemplate="Fecha: %{x}<br>Caudal: %{y} L/s"
))

# Agregar la línea de presión (verde) en el eje derecho
fig.add_trace(go.Scatter(
    x=df_presion["FECHA"], y=df_presion["VALUE"], mode="lines+markers",
    name="Presión (Kg/cm²)", line=dict(color="green"), marker=dict(size=6),
    yaxis="y2", hovertemplate="Fecha: %{x}<br>Presión: %{y} Kg/cm²"
))

# Personalizar el gráfico
fig.update_layout(
    title=" P002 Caudal y Presión",
    xaxis=dict(title="Fecha", tickangle=45),
    yaxis=dict(title="Caudal (L/s)", side="left"),
    yaxis2=dict(title="Presión (Kg/cm²)", overlaying="y", side="right"),
    hovermode="x unified",
    template="plotly_dark"
)

# Mostrar el gráfico
fig.show()

pio.write_html(fig, file="grafico_caudal_presion.html", auto_open=True)